In [48]:
import datetime as dt
from steemdata import SteemData

from funcy.colls import pluck
from funcy.seqs import mapcat
from funcy.flow import silent

In [2]:
from steem.post import Post

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
db = SteemData().db

In [31]:
time_constraints = {
    '$gte': dt.datetime.now() - dt.timedelta(days=7),
}
conditions = {
    'created': time_constraints,
}
projection = {
    '_id': 0,
    'identifier': 1,
    'abs_rshares': 1,
    'net_rshares': 1,
    'vote_rshares': 1,
    'children_abs_rshares': 1,
}
posts = list(db['Posts'].find(conditions, projection=projection))

In [32]:
## get the comments

In [33]:
identifiers = list(pluck('identifier', posts))

In [34]:
len(identifiers)

10145

In [13]:
def get_comments(identifier):
    try:
        p = Post(identifier)
    except:
        return []
    return Post.get_all_replies(p)

In [51]:
posts = map(silent(Post), identifiers)
# remove empty ones
posts = list(filter(bool, posts))

In [14]:
comments = mapcat(get_comments, identifiers)

In [18]:
## tally up rshares

In [58]:
def vote_rshares_sum(post):
    return sum(map(int, pluck('rshares', post.active_votes)))

In [59]:
def rshares_sum(posts):
    return sum(map(vote_rshares_sum, posts))

In [60]:
post_rshares = rshares_sum(posts)

In [70]:
comment_rshares = rshares_sum(comments)

In [ ]:
comment_rshares / post_rshares * 100